# Introducción

Este cuaderno hace parte de la prueba técnica en el proceso de selección de Cientifico(a) de Datos del Banco Davivienda.

En la construcción de este cuaderno se tomaron los siguientes referentes:

1. [Cuaderno del diplomado de aprendizaje profundo de la Universidad Nacional de Colombia](https://nbviewer.org/github/AprendizajeProfundo/PLN/blob/main/Presentaciones/pln_sesion_01_01_introduccion.ipynb)
2. [Artículo de Sarkar (2018)](https://towardsdatascience.com/understanding-feature-engineering-part-4-deep-learning-methods-for-text-data-96c44370bbfa)
3. [Curso de Hugging Face](https://huggingface.co/course/chapter1/3?fw=pt)

# Importar paquetes

In [68]:
# Drive
from google.colab import drive, files
drive.mount('/content/drive')

# Básicos
import numpy as np
import pandas as pd
import datetime
import itertools # Para obtener productos cartesianos
pd.options.display.max_columns = None # Presentación de los dataframes

# Gráficos
!pip install plotly==5.5.0
import plotly.graph_objects as go
import plotly.express as px
import xarray as xr
#
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Tiempo
from datetime import datetime

### PLN
# Hugging Face
!pip install datasets evaluate transformers[sentencepiece]
from transformers import pipeline
translator = pipeline("translation", model="Helsinki-NLP/opus-mt-es-en")
classifier = pipeline("sentiment-analysis")
# Gensim
from gensim.models import word2vec
# Limpiar texto
!pip install stanza
import stanza # Lematizar
nlp = stanza.Pipeline(lang='es', processors='tokenize,mwt,pos,lemma')
import re # Eliminar caracteres especiales
# Stop words
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
stop_nltk = stopwords.words('spanish') # Stop word de nltk
import spacy
!python -m spacy download es_core_news_sm
import es_core_news_sm
nlp_spacy = es_core_news_sm.load()
stop_spacy = nlp_spacy.Defaults.stop_words # Stop words de spacy
stop_todas = list(stop_spacy.union(set(stop_nltk))) # Stop words

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.9/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning:

Recommended: pip install sacremoses.

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.9/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-03-20 18:34:33.156855: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-20 18:34:33.157123: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-03-20 18:34:33.157154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

# Tomar datos

In [69]:
directorio = '/content/drive/MyDrive/Colab Notebooks/TestD'
archivo_nom = 'davivienda_tweets.csv'
ruta = [directorio, archivo_nom]
ruta = '/'.join(ruta)
datos = pd.read_csv(ruta, delimiter=",")
datos

,Unnamed: 0,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL
0,0,Andrés Langebaek,@ALangebaek,2021-12-01T20:43:12.000Z,Andrés Langebaek\n@ALangebaek\n·\n1 dic.,La confianza se afectó. El indicador de confia...,NaN,1.0,7,19,['https://pbs.twimg.com/media/FFjL57eXMAISBnk?...,https://twitter.com/ALangebaek/status/14661458...
1,1,Plaza Futura,@plaza_futura,2021-12-01T21:18:10.000Z,Plaza Futura\n@plaza_futura\n·\n1 dic.,Buscamos la accesibilidad y mejor atención en ...,✅ ✅ ✅ ✅ ✅,NaN,NaN,NaN,['https://pbs.twimg.com/ext_tw_video_thumb/146...,https://twitter.com/plaza_futura/status/146615...
2,2,Julián Martinez,@JulianM998,2021-12-01T22:49:11.000Z,Julián Martinez\n@JulianM998\n·\n1 dic.,Señores \n@Davivienda\n no he podido ingresar ...,NaN,1.0,NaN,1,[],https://twitter.com/JulianM998/status/14661775...
3,3,Ferchis.,@fergomezr28,2021-12-01T12:29:07.000Z,Ferchis.\n@fergomezr28\n·\n1 dic.,Llevo toda una semana sufriendo intento de hur...,NaN,2.0,1,2,[],https://twitter.com/fergomezr28/status/1466021...
4,4,MirandaL2,@MirandaSuspLo,2021-12-01T20:52:36.000Z,MirandaL2\n@MirandaSuspLo\n·\n1 dic.,Hemos retrocedido tanto en este país con este ...,NaN,3.0,NaN,8,[],https://twitter.com/MirandaSuspLo/status/14661...
...,...,...,...,...,...,...,...,...,...,...,...,...
1806,1806,Banco Davivienda,@Davivienda,2021-12-22T18:26:38.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,En respuesta a \n@JaimeMolina\nBuenas tardes. ...,NaN,1.0,NaN,NaN,[],https://twitter.com/Davivienda/status/14737216...
1807,1807,Banco Davivienda,@Davivienda,2021-12-22T20:18:40.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,"En respuesta a \n@josefe71\nHola Jose , gracia...",NaN,1.0,NaN,NaN,[],https://twitter.com/Davivienda/status/14737498...
1808,1808,Banco Davivienda,@Davivienda,2021-12-22T17:54:45.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,En respuesta a \n@gleniiaaa\nHola! Bienvenida ...,👋,NaN,NaN,NaN,[],https://twitter.com/Davivienda/status/14737136...
1809,1809,Banco Davivienda,@Davivienda,2021-12-22T11:45:13.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,"En respuesta a \n@diegoruix\nBuenos días, Sr. ...",NaN,NaN,NaN,NaN,[],https://twitter.com/Davivienda/status/14736206...


# Análisis descriptivo

Identificación simple de algunos patrones:



1.   Al final del texto hay un concatenado del número de comentarios, likes y retweets.
2.   Nombrar un usuario se hace en una línea aparte, ejemplo, \n@XXX\n
3. Los Tweets se realizaron entre el primero y el veintidós de diciembre de 2021.


In [70]:
print('Tweet más antiguo:', datos['Timestamp'].min())
print('Tweet más reciente:', datos['Timestamp'].max())

Tweet más antiguo: 2021-12-01T00:01:49.000Z
Tweet más reciente: 2021-12-22T23:53:50.000Z


A continuación se muestra que la gran mayoría de los usuarios tienen sólo un Tweet. Los datos atípicos más relevantes son dos cuentas institucionales de Davivienda (@Davivienda y @davicorredores) y un usuario (@CNOGUERA20) con 41 Tweets.

In [71]:
datos['tweets'] = 1
ntw_xuser = datos.groupby(['UserName'], as_index=False)['tweets'].sum()
print('Número de Tweets en la base de datos\n', ntw_xuser['tweets'].sum())
print('Número de usuarios en la base de datos\n', ntw_xuser['tweets'].shape[0])
print('\n')
fig = px.box(ntw_xuser, y='tweets', hover_data=['UserName'], template='plotly_white', 
             title='Boxplot del número de Tweets por usuario', labels={'tweets':'Tweets'}, 
             width=700, height=500)
fig.show()

Número de Tweets en la base de datos
 1811
Número de usuarios en la base de datos
 1168




Se identifica Tweets de "En respuesta a..." o "Respondiendo a...", y se establece su relación con el usuario, que aquí resumiremos en las cuentas institucionales de Davivienda y el resto de usuarios.

Se encuentra que la mayor parte de los Tweets son "En respuesta a..." o "Respondiendo a...". Igualmente, la cuenta de @Davivienda se caracteriza por responder a otros usuarios, la cuenta de @davicorredores se caracteriza por generar Tweets propios, y en el resto de usuarios hay una magnitud similar entre Tweets propios y "En respuesta a...".

In [72]:
# Tipo de Tweet
datos['tweet_type'] = datos['Embedded_text'].apply(lambda row: 'Propio' if row.find('\n@')==-1 else row[0:row.find('\n@')].strip())
datos.loc[~datos['tweet_type'].isin(['En respuesta a', 'Respondiendo a']), 'tweet_type'] = 'Propio'
print('La mayoría de los Tweets son "En repuesta a..." o "Respondiendo a..."\n',datos.groupby(['tweet_type'], as_index=False)['tweets'].sum())

# Tipo de usuario
datos['user_type'] = datos['UserName']
datos.loc[~datos['UserName'].isin(['@Davivienda', '@davicorredores']), 'user_type'] = '@Otro'
print('\nLa mayoría de los Tweets no son institucionales\n',datos.groupby(['user_type'], as_index=False)['tweets'].sum())

# Relación entre el tipo de usuario y el tipo de Tweet
print('\n')
fig = px.bar(datos.groupby(['user_type', 'tweet_type'], as_index=False)['tweets'].sum(), x='user_type', y='tweets', color='tweet_type', 
             template='plotly_white', barmode='group',
             title='Tweets según tipo de usuario y tipo de Tweet', labels={'tweets':'Número de Tweets', 'user_type':'Tipo de usuario'}, 
             width=900, height=500)
fig.show()

La mayoría de los Tweets son "En repuesta a..." o "Respondiendo a..."
        tweet_type  tweets
0  En respuesta a    1001
1          Propio     677
2  Respondiendo a     133

La mayoría de los Tweets no son institucionales
          user_type  tweets
0      @Davivienda     245
1            @Otro    1543
2  @davicorredores      23




# Análisis de PLN

Ajustamos levemente el texto de los Tweets para eliminar el patrón al final de comentarios, likes y retweets y quitamos las expresiones "\n"

In [73]:
for i in ['Comments', 'Likes', 'Retweets']:
  datos[i] = datos[i].astype(str)
  datos[i] = datos[i].str.replace('.0', '')
datos['patron'] = '\n' + datos['Comments'] + '\n' + datos['Likes'] + '\n' + datos['Retweets']
datos['patron'] = datos['patron'].str.replace('\nnan', '')
datos['Embedded_text'] = datos.apply(lambda x: x['Embedded_text'].replace(x['patron'], ''), axis=1)
datos['Embedded_text'] = datos['Embedded_text'].str.replace('\n', '')
datos

<ipython-input-73-bbaca1a11445>:3: FutureWarning:

The default value of regex will change from True to False in a future version.



,Unnamed: 0,UserScreenName,UserName,Timestamp,Text,Embedded_text,Emojis,Comments,Likes,Retweets,Image link,Tweet URL,tweets,tweet_type,user_type,patron
0,0,Andrés Langebaek,@ALangebaek,2021-12-01T20:43:12.000Z,Andrés Langebaek\n@ALangebaek\n·\n1 dic.,La confianza se afectó. El indicador de confia...,NaN,1,7,19,['https://pbs.twimg.com/media/FFjL57eXMAISBnk?...,https://twitter.com/ALangebaek/status/14661458...,1,Propio,@Otro,\n1\n7\n19
1,1,Plaza Futura,@plaza_futura,2021-12-01T21:18:10.000Z,Plaza Futura\n@plaza_futura\n·\n1 dic.,Buscamos la accesibilidad y mejor atención en ...,✅ ✅ ✅ ✅ ✅,nan,nan,nan,['https://pbs.twimg.com/ext_tw_video_thumb/146...,https://twitter.com/plaza_futura/status/146615...,1,Propio,@Otro,
2,2,Julián Martinez,@JulianM998,2021-12-01T22:49:11.000Z,Julián Martinez\n@JulianM998\n·\n1 dic.,Señores @Davivienda no he podido ingresar a mi...,NaN,1,nan,1,[],https://twitter.com/JulianM998/status/14661775...,1,Propio,@Otro,\n1\n1
3,3,Ferchis.,@fergomezr28,2021-12-01T12:29:07.000Z,Ferchis.\n@fergomezr28\n·\n1 dic.,Llevo toda una semana sufriendo intento de hur...,NaN,2,1,2,[],https://twitter.com/fergomezr28/status/1466021...,1,Propio,@Otro,\n2\n1\n2
4,4,MirandaL2,@MirandaSuspLo,2021-12-01T20:52:36.000Z,MirandaL2\n@MirandaSuspLo\n·\n1 dic.,Hemos retrocedido tanto en este país con este ...,NaN,3,nan,8,[],https://twitter.com/MirandaSuspLo/status/14661...,1,Propio,@Otro,\n3\n8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1806,1806,Banco Davivienda,@Davivienda,2021-12-22T18:26:38.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,En respuesta a @JaimeMolinaBuenas tardes. Sr. ...,NaN,1,nan,nan,[],https://twitter.com/Davivienda/status/14737216...,1,En respuesta a,@Davivienda,\n1
1807,1807,Banco Davivienda,@Davivienda,2021-12-22T20:18:40.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,"En respuesta a @josefe71Hola Jose , gracias po...",NaN,1,nan,nan,[],https://twitter.com/Davivienda/status/14737498...,1,En respuesta a,@Davivienda,\n1
1808,1808,Banco Davivienda,@Davivienda,2021-12-22T17:54:45.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,En respuesta a @gleniiaaaHola! Bienvenida a nu...,👋,nan,nan,nan,[],https://twitter.com/Davivienda/status/14737136...,1,En respuesta a,@Davivienda,
1809,1809,Banco Davivienda,@Davivienda,2021-12-22T11:45:13.000Z,Banco Davivienda\n@Davivienda\n·\n22 dic.,"En respuesta a @diegoruixBuenos días, Sr. Dieg...",NaN,nan,nan,nan,[],https://twitter.com/Davivienda/status/14736206...,1,En respuesta a,@Davivienda,


## Identificar el sentimiento de los Tweets

Usamos transformers de la libreria de Huging Face para identificar el grado de sentimiento positivo o negativo de los Tweets. 

1. Traducimos los Tweet a inglés.
2. Identificamos su índice de sentimiento negativo o positivo.
3. Agregamos este índice a la base de datos.

Se aprecia que el tiempo de obtener el sentimiento de los Tweets es considerable aún usando con TPU como entorno de ejecución. Sólo se obtiene el sentimiento de los primeros 100 Twweets.

In [74]:
# Traducir y obtener el sentimieno de cada Tweet
ini = datetime.now()
trad = translator(list(datos['Embedded_text'])[0:100])
trad = list(pd.DataFrame(trad)['translation_text'].values)
feel = classifier(trad)
feel = pd.DataFrame(feel)
fin = datetime.now()

# Tiempo que tardó
diff = fin - ini
print('La identificación del sentimiento de los primeros 100 Tweets tardó ', round(diff.total_seconds(), 0), ' segundos')

# Agregar sentimientos la base
datos = pd.concat([datos, feel], ignore_index=False, axis=1)

La identificación del sentimiento de los primeros 100 Tweets tardó  569.0  segundos


## Identificar temáticas de los Tweet mediante embedings

Normalizar los textos a pasar por el embeding:

1. Quitar caracteres especiales
2. Dejar en minísculas
3. Quitar vacios
4. Tokenizar por palabra
5. Quitar palabras vacías
6. No se lematiza por tiempo de procesamiento

In [75]:
# Textos a normalizar tokenizando por palabra
textos = np.array(datos['Embedded_text'])
wpt = nltk.WordPunctTokenizer()

# Función que aplica a cada texto
def normalize_document(doc):
    doc = re.sub(r'[^\w\s]', '', doc, re.I|re.A) 
    doc = doc.lower()
    doc = doc.strip()
    tokens = wpt.tokenize(doc)
    filtered_tokens = [token for token in tokens if token not in stop_todas]
    doc = ' '.join(filtered_tokens)
    #doc = ' '.join([lema.lemma for lema in nlp(doc).sentences[0].words])
    return doc

# crea una función vectorizada para que actué sobre múltiples textos
normalize_corpus = np.vectorize(normalize_document)
textos_nor = normalize_corpus(textos)

# Normalizar textos
textos_nor = pd.DataFrame(textos_nor)
textos_nor.columns = ['texto_nor']

# Agregar textos normalizados a la base
datos = pd.concat([datos, textos_nor], ignore_index=False, axis=1)

Se obtienen embedings usando Gensim. Se elige una dimensión de de 2 para el embeding para poder tener una presentación directada en un plano cartesiano de las palabras y los Tweets.

In [76]:
# Texto normalizado para Gensim
texto_nor_g = [wpt.tokenize(tweet) for tweet in list(datos['texto_nor'])]

# Establecer parámetros para el embeding
feature_size = 2
window_context = 10
min_word_count = 1
sample = 1e-3

# Embeding
w2v_model = word2vec.Word2Vec(texto_nor_g, size=feature_size, window=window_context, min_count=min_word_count, sample=sample, iter=500)

# Obtener la palabras
words = w2v_model.wv.index2word
words_df = pd.DataFrame(words)
words_df.columns = ['word']
words_df

# Agregrar ocurrencia de cada palabra
all_words = pd.DataFrame(' '.join(list(datos['texto_nor'])).split(' '))
all_words.columns = ['word']
all_words['ocurrencias'] = 1
all_words = all_words.groupby(['word'], as_index=False)['ocurrencias'].sum()
all_words
# Agregar al data frame del embeding de palabras
words_df = words_df.merge(all_words, on=['word'], how='left', validate='1:1')
words_df

# Obtener embeding de las palabras
embe = w2v_model.wv[words]
embe = pd.DataFrame(embe)
embe.columns = ['dim1', 'dim2']
embe = pd.concat([embe, words_df], ignore_index=False, axis=1)
embe

,dim1,dim2,word,ocurrencias
0,-0.696223,0.512372,davivienda,1122
1,-0.638265,-0.351404,respuesta,1065
2,-3.970030,-3.829503,mensaje,285
3,-6.927981,-6.543627,privado,231
4,-1.857676,-1.885007,favor,227
...,...,...,...,...
7287,-1.138194,-0.340877,josefe71hola,1
7288,-2.250073,-1.127304,gleniiaaahola,1
7289,-2.778753,-2.098642,acompañaré,1
7290,-0.848301,-0.414873,diegoruixbuenos,1


Embedings para cada Tweet. Se obtienen los embedings de cada Tweet promediando los embedings de las palabras con que lo componen.

In [77]:
# Funciones que obtienen el embeding de los Tweets promediando los embedings de las palabras que lo contienen
def average_word_vectors(words, model, vocabulary, num_features):
    feature_vector = np.zeros((num_features,),dtype="float64")
    nwords = 0.
    for word in words:
        if word in vocabulary: 
            nwords = nwords + 1.
            feature_vector = np.add(feature_vector, model[word])
    if nwords:
        feature_vector = np.divide(feature_vector, nwords)
    return feature_vector
def averaged_word_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

# recupera el incrustamientoa nivel de documento
embe_tw = averaged_word_vectorizer(corpus=texto_nor_g, model=w2v_model, num_features=feature_size)                        
embe_tw = pd.DataFrame(embe_tw)
embe_tw.columns = ['dim1', 'dim2']
embe_tw

# Agregrar información a los embedings de los Tweets
embe_tw = pd.concat([embe_tw, datos[['tweet_type', 'user_type', 'Embedded_text', 'label', 'score']]], ignore_index=False, axis=1)
embe_tw.loc[embe_tw['label'].isnull(), 'label'] = 'No calculado'
embe_tw.loc[embe_tw['score'].isnull(), 'score'] = 0
embe_tw

<ipython-input-77-4b30840c34b7>:8: DeprecationWarning:

Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).



,dim1,dim2,tweet_type,user_type,Embedded_text,label,score
0,-1.072108,1.989940,Propio,@Otro,La confianza se afectó. El indicador de confia...,NEGATIVE,0.999107
1,-1.551301,-0.630819,Propio,@Otro,Buscamos la accesibilidad y mejor atención en ...,NEGATIVE,0.990945
2,-1.228668,2.762343,Propio,@Otro,Señores @Davivienda no he podido ingresar a mi...,NEGATIVE,0.972933
3,-1.156281,2.160152,Propio,@Otro,Llevo toda una semana sufriendo intento de hur...,NEGATIVE,0.999433
4,-1.040142,2.342584,Propio,@Otro,Hemos retrocedido tanto en este país con este ...,NEGATIVE,0.996569
...,...,...,...,...,...,...,...
1806,-4.673758,-4.259300,En respuesta a,@Davivienda,En respuesta a @JaimeMolinaBuenas tardes. Sr. ...,No calculado,0.000000
1807,-6.177930,-5.738940,En respuesta a,@Davivienda,"En respuesta a @josefe71Hola Jose , gracias po...",No calculado,0.000000
1808,-4.826586,-4.431797,En respuesta a,@Davivienda,En respuesta a @gleniiaaaHola! Bienvenida a nu...,No calculado,0.000000
1809,-6.686968,-6.248569,En respuesta a,@Davivienda,"En respuesta a @diegoruixBuenos días, Sr. Dieg...",No calculado,0.000000


Análisis de los embedings.

En primer lugar la mayor parte de la palabras tiene una ocurrencia en la base de datos de hasta tres veces. La palabra con mayor ocurrencia en "Davivienda", como era de esperarse.

In [78]:
fig = px.box(embe, y='ocurrencias', hover_data=['word'], template='plotly_white', 
             title='Boxplot del número de ocurrencia de la palabra', labels={'ocurrencias':'Ocurrencia'}, 
             width=700, height=500)
fig.show()

Se obtiene una representación númerica de las palabras enun plano cartesiano. La cercanía de las palabras permite identificar palabras que suelen usarse en conjunto, con lo cual es una herramienta de la que se puede inferir temáticas que se encuentran en los Tweet.

In [79]:
fig = px.scatter(embe.loc[(embe['ocurrencias']>10), ], x='dim1', y='dim2', size='ocurrencias', hover_data=['word', 'ocurrencias'],
                 marginal_x='histogram', marginal_y='histogram',
                 title='Representación númerica de las palabras en un plano cartesiano', labels={'dim1':'Dimensión 1', 'dim2':'Dimensión 2'},
                 template='plotly_white', width=900, height=800, size_max=30)
fig.show()

Así mismo, promediando las coordenadas de cada una de las palabras que conforman los Tweets se puede obtener una representación en el plano cartesiano de los Tweets.

Se aprecia como el patrón de témáticas de cuentas institucionales de Davivienda es claramente distinto al patron de temáticas del resto de los usuarios.

In [80]:
fig = px.scatter(embe_tw, x='dim1', y='dim2', color='user_type', hover_data=['Embedded_text'],
                 marginal_x='histogram', marginal_y='histogram',
                 title='Representación númerica de los Twets en un plano cartesiano, según tipo de usuario', labels={'dim1':'Dimensión 1', 'dim2':'Dimensión 2'},
                 template='plotly_white', width=900, height=800)
fig.show()

En el plano también se pueden representar los Tweet según el sentimiento positivo o negativo que se identificó de ellos arriba (sólo se obtuvo para los primeros 100 Tweet).

Se encuentra que los Tweets positivos tienen a ubicarse a la derecha y abajo del plano, y los negativos a la izquiera arriba. Superponiendo la ubicación de los Tweets positivos con los negativos y usando como apoyo métodos de cluster se pueden identificar temáticas asociadas a sentimientos negativos o positivos.

In [81]:
fig = px.scatter(embe_tw.loc[~embe_tw['label'].isin(['No calculado']), ], x='dim1', y='dim2', color='label', hover_data=['Embedded_text'],
                 marginal_x='histogram', marginal_y='histogram',
                 title='Representación númerica de los Twets en un plano cartesiano, según sentimiento', labels={'dim1':'Dimensión 1', 'dim2':'Dimensión 2'},
                 template='plotly_white', width=900, height=800)
fig.show()